# Generative chatbot

## step1 import lib

In [1]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

## step2 load datasets

In [3]:
ds = Dataset.load_from_disk(r"./alpaca_data_zh")
ds

Dataset({
    features: ['output', 'input', 'instruction'],
    num_rows: 26858
})

In [7]:
ds[1:4]

{'output': ['4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。',
  '朱利叶斯·凯撒，又称尤利乌斯·恺撒（Julius Caesar）是古罗马的政治家、军事家和作家。他于公元前44年3月15日被刺杀。 \n\n根据历史记载，当时罗马元老院里一些参议员联合起来策划了对恺撒的刺杀行动，因为他们担心恺撒的统治将给罗马共和制带来威胁。在公元前44年3月15日（又称“3月的艾达之日”），恺撒去参加元老院会议时，被一群参议员包围并被攻击致死。据记载，他身中23刀，其中一刀最终致命。',
  '法国的首都是巴黎。'],
 'input': ['输入：4/16', '', ''],
 'instruction': ['解释为什么以下分数等同于1/4', '朱利叶斯·凯撒是如何死亡的？', '法国的首都是什么？']}

## step3 data process

In [8]:
model_path = r"D:\CodeLibrary\huggingface_model\Langboat\bloom-389m-zh"
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
tokenizer

BloomTokenizerFast(name_or_path='D:\CodeLibrary\huggingface_model\Langboat\bloom-389m-zh', vocab_size=42437, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

这里 `token` 最大长度为256， 是因为之前的数据特殊处理过，最长不会超过256，实际开发时，看数据情况调整即可

`[-100] * len(instruction['input_ids'])` 是因为只做 `output` 的loss计算

In [13]:
def process_func(example):
    MAX_LENGTH = 256
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human: "+ example["instruction"], example['input']]).strip() + "\n\nAssistant: ")
    response = tokenizer(example['output'] + tokenizer.eos_token)

    # 拼接
    input_ids = instruction['input_ids'] + response['input_ids']
    attention_mask = instruction['attention_mask'] + response['attention_mask']
    labels = [-100] * len(instruction['input_ids']) + response['input_ids']

    # 截断
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [14]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

Map:   0%|          | 0/26858 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 26858
})

In [15]:
tokenizer.decode(tokenized_ds[1]['input_ids'])

'Human: 解释为什么以下分数等同于1/4\n输入：4/16\n\nAssistant: 4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。</s>'

In [16]:
# decode看下 `labels`
tokenizer.decode(list(filter(lambda x : x != -100, tokenized_ds[1]['labels'])))

'4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。</s>'

## step4 create model

In [17]:
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)

e:\software\anaconda3\envs\transformers\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


## step5 config train parameters

In [18]:
args = TrainingArguments(
    output_dir="./chatbot",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    logging_steps=10,
    num_train_epochs=2
)

## step6 create trainer

In [19]:
trainer = Trainer(
    model=model,
    args=args,
    tokenizer=tokenizer,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True)
)

## step7 model train

In [20]:
trainer.train()

Step,Training Loss
10,3.172300
20,3.223200
30,3.108100
40,3.146000
50,3.070500
60,2.972200
70,2.924700
80,2.852000
90,2.777400
100,2.910200


TrainOutput(global_step=1678, training_loss=2.108726101921909, metrics={'train_runtime': 2561.3754, 'train_samples_per_second': 20.972, 'train_steps_per_second': 0.655, 'total_flos': 1.164486099812352e+16, 'train_loss': 2.108726101921909, 'epoch': 1.9991064780342516})

## step8 model inference

In [21]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

In [24]:
ipt = "Human: {}\n{}".format("考试有哪些技巧？", "").strip() + "\n\nAssistant: "
text = pipe(ipt, max_length=256, do_sample=True, )
text

[{'generated_text': 'Human: 考试有哪些技巧？\n\nAssistant: 考试技巧指在经过充分的准备和资源的情况下，能够运用所学知识，以达到考试目的的一项智力活动。以下是一些常见的考试技巧可能用的到的：\n\n1. 组织考试前确保考室整洁明亮：考前组织考试时，应确保考室内尘埃是否均已清除干净，以免在考试过程中滋生细菌，细菌会影响考试，最好在空旷、安静的考室中进行。\n\n2. 注意检查考试准错率：考试准错率是考试质量的一个重要体现，过多的或不充分的准备工作可能会导致考试出现错误。应注意检查每个考点的准确性，并记录在试卷上，以避免在考试过程中出现错误的记录。\n\n3. 注意放松，认真思考：在考试过程中要保持头脑中思维平静，在完成考试的同时集中精神集中注意力，不要分心，能有效提高考试的效率。\n\n总的来说，考试技巧对于提高学习效率，确保考试结果的重要性和重要性不言而喻。'}]

In [27]:
ans = text[0]['generated_text']
print(ans)

Human: 考试有哪些技巧？

Assistant: 考试技巧指在经过充分的准备和资源的情况下，能够运用所学知识，以达到考试目的的一项智力活动。以下是一些常见的考试技巧可能用的到的：

1. 组织考试前确保考室整洁明亮：考前组织考试时，应确保考室内尘埃是否均已清除干净，以免在考试过程中滋生细菌，细菌会影响考试，最好在空旷、安静的考室中进行。

2. 注意检查考试准错率：考试准错率是考试质量的一个重要体现，过多的或不充分的准备工作可能会导致考试出现错误。应注意检查每个考点的准确性，并记录在试卷上，以避免在考试过程中出现错误的记录。

3. 注意放松，认真思考：在考试过程中要保持头脑中思维平静，在完成考试的同时集中精神集中注意力，不要分心，能有效提高考试的效率。

总的来说，考试技巧对于提高学习效率，确保考试结果的重要性和重要性不言而喻。
